# DBAP CA2
## Jia Lin 22117644 teamC
## TABLE 3: The US Confirmed and Death by Provinces
### Import Necessary Libraries

In [85]:
import requests             # library request is used to fetch data from API 
import json                 # library json is used load json objects
import datetime             # library datetime is used to convert between datetime and string
from datetime import date
import pymongo              # library pymongo is used to build connection with MongoDB
import pandas               # library pandas is used to create a ETL pipeline
import psycopg2             # library psycopg2 is used to make connection with PostgreSQL
import csv                  # library csv is used to store csv to PostgreSQL

### Function Pool

In [86]:
# getResponse function is defined to get response from web "url" based on "query"
def getResponse(url, query):
    return requests.request("GET", url, headers=headers, params=query)

# getJSON_obj function is defined to get a json object based on "response"
def getJSON_obj(response):
    return json.loads(response.content.decode("utf-8"))

# get_data_value function is defined to return the value of key data
def get_data_value(url, query):
    json_obj = getJSON_obj(getResponse(url, query))["data"]
    #if len(json_obj) == 0: 
     #   print("Notice: there is no data on this date!")
    return json_obj

# get_data_values function is defined to get a list of json values of key data
# For each json_obj, there is a json_obj["data"]
# This function will return a list of json_obj["data"]
# based on a list of queries from a specific url
def get_data_values(url, query_list):
    return [get_data_value(url, q) for q in query_list]

# a_day_before function is defined to get the date of one day before the day_date
def a_day_before(day_date): 
    return day_date - datetime.timedelta(days=1)

# str_to_date function is defined to convert a string to a date
def str_to_date(s): 
    return datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S').date()

# date_to_str function is defined to convert a date to a string
def date_to_str(dt): 
    return dt.strftime('%Y-%m-%d')  

# get_date_list function is defined to return a list of date in string format
# by given the start_date and the number of days
def get_date_list(start_date, number_of_days):
    date_datetype_list = [start_date - datetime.timedelta(days=x) for x in range(number_of_days)]
    return [date_to_str(x) for x in date_datetype_list]

def get_csv(collection_name, csv_file_name):
    cursors = collection_name.find()
    # Convert the mongo documents to a DataFrame
    documents = pandas.DataFrame(cursors)
    # Discard the Mongo ID for the documents
    documents.pop("_id")
    # export MongoDB documents to a csv file, leaving out the row "labels" (row numbers)
    documents.to_csv(csv_file_name, ",", index=False)

## ETL pipeline: Extract
### Fetch Data from API (Extract)
#### Two Queries:
#### 1) Query for US Provinces or State
#### 2) Query for Covid-19 Data in Cities of Each Province

In [87]:
# Source code: https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/
# Based on public data by Johns Hopkins CSSE
# In terms of the requirements of this project, Jia Lin made some modifications on the source code
headers = {"X-RapidAPI-Key": "bc72ee8736mshfc960795af3b6ddp15d851jsn50353bd76099",
           "X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"}

# List of provinces by country ISO code.
url_provinces = "https://covid-19-statistics.p.rapidapi.com/provinces"
query_iso = {"iso":"USA"}
# Original data from API 
provinces = get_data_value(url_provinces, query_iso)
#print(provinces[0])

# Reports by date an country/province. 
# Cities data is available for the USA only.
url_US_reports = "https://covid-19-statistics.p.rapidapi.com/reports"
query_US_reports = {"iso":"USA","date":"2022-12-06"}
# In this query, the date is an vital parameter
# Normally, the latest data is a day before current date
data_value_US_reports = get_data_value(url_US_reports, query_US_reports)

## ETL pipeline: Transform
### Get Covid-19 Info for All US Province
#### The aim of this section is clean and transform the data
#### The instance will not be collected, when meet any issue below:
#### 1) If the province itself has no value to present.
#### 2) If the province has no cities info to show
#### 3) If some irrelevant attributes are found, for eample, wrong city name, empty value (None)

In [88]:
# Create a list of queries with different province
# Be careful with the date
def get_query_province(province):
    return {"region_province": province ,"iso":"USA", "date":"2022-12-06"}

provinces_name_list = [p["province"] for p in provinces]

no_match_provinces = []# province in this list has no info to present
matching_provinces = []# province in this list contain data to present
provinces_info_list = []# a list of list of cities of each province
for province_name in provinces_name_list:
    # report is a list of length of 0 or 1
    # when length is 0 indicates no match province info 
    report = get_data_value(url_US_reports, get_query_province(province_name))
    if len(report) == 0:# this province has no info
        no_match_provinces.append(province_name)
        continue
    if len(report) == 1:  
        kv = {"province": report[0]["region"]["province"]}
        cities = report[0]["region"]["cities"]
        if len(cities) == 0:# this province has no cities info
            no_match_provinces.append(province_name)
        else:
            matching_provinces.append(province_name)
            for city in cities: 
                if city['name'] == 'Unassigned' or city['lat'] is None or city['fips'] is None:
                    continue
                else: 
                    city.update(kv)
                    provinces_info_list.append(city)
    else: print("There is an error, the len is eigher 1 or 0.")

############################################################################################
# There are two kinds of provinces in this no_match_province
# One is this province has no info
# The other one is this province has no cities info
############################################################################################
# Clean data provinces
# Only the provinces with data will be maintained
def isMatchingProvinces(str):
    for name in matching_provinces:
        if str == name:
            return True
    return False

new_provinces = []
for item in provinces:
    if isMatchingProvinces(item["province"]):
        new_provinces.append(item)
    else: continue
# This new_provinces will be stored in MongoDB
#print(new_provinces)

### Making a Connection with MongoDB
#### Two collections related to provinces
#### 1) provinces_of_US_collection
#### 2) reports_US_provinces_cities_collection

In [89]:
# Source code: DBAP_Lab_Week6 (MogoDB)
client = pymongo.MongoClient('localhost', 27017)
# create a database called jialin_Mongo_database (may change to covid19JHCSSE_database)
db = client.jialin_Mongo_database
# create a cellections
provinces_of_US_collection = db.provinces_of_US_collection
reports_US_provinces_cities_collection = db.reports_US_provinces_cities_collection

provinces_of_US_collection.drop()
reports_US_provinces_cities_collection.drop()

# Insert more than 1 docuement using insert_many method
# collection 3
# Note that it is new_provinces list
provinces_of_US_collection.insert_many(new_provinces)
# collection 4
reports_US_provinces_cities_collection.insert_many(provinces_info_list)

database_list = client.list_database_names()
print ("db names:", database_list)
db.list_collection_names()

db names: ['admin', 'config', 'jialin_Mongo_database', 'local', 'test_database']


['global_date_over_time_collection',
 'provinces_of_US_collection',
 'reports_US_data_over_time_collection',
 'reports_US_provinces_cities_collection']

## ETL pipeline: Transform
### JSON to CSV
### Export CSV File from MongoDB

In [90]:
get_csv(provinces_of_US_collection, "DBAP_CA2_provinces_US.csv")
get_csv(reports_US_provinces_cities_collection, "DBAP_CA2_reports_US_provinces_cities.csv")